k-최근접 이웃 분류기는 기술적으로 예측을 만들기 위해 모델을 훈련하지 않는다. 대신에 가장 가까운 k 개의 샘플에서 다수의 클래스를 그 샘플의 클래스로 예측한다 

### 샘플의 최근접 이웃 찾기
샘플에서 가장 가까운 k개의 샘플(이웃)을 찾아야 한다. -> 사이킷런의 NearestNeighbors를 사용한다.

In [17]:
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data

scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

nearest_neighbors = NearestNeighbors(n_neighbors=2).fit(features_standardized)

new = [ 1, 1, 1, 1]

# new 샘플과 가까운 이웃의 거리와 인덱스를 찾는다.
distances, indices = nearest_neighbors.kneighbors([new])
neighbors = features_standardized[indices]

print(neighbors, distances)

[[[1.03800476 0.55861082 1.10378283 1.18556721]
  [0.79566902 0.32841405 0.76275827 1.05393502]]] [[0.49140089 0.74294782]]


- 거리를 측정하는 원리: 사이킷런은 유클리드, 맨해튼, 민코프스키 거리를 사용. NearestNeighbors는 민코프스키 거리를 기본으로 한다.
    - $d_{euclidean} = \sqrt{\sum^{n}_{i=1}{(x_i-y_i)}^2}$
    - $d_{manhatton} = \sum^{n}_{i=1}{|x_i-y_i|}$
    - $d_{minkowski} = ( \sum^n_{i=1} |x_i-y_i|^p)^{\frac{1}{p}}$
(민코프스기 거리에는 하이퍼 파라미터 p가 있다. p=1이면 맨해튼 거리. p=2이면 유클리드 거리가 된다. 사이킷런의 기본값은 p=2이다.)


- 측정 방법은 metirc 매개변수를 이용해 지정할 수 있다.

In [18]:
nearestneighbors_euclidean = NearestNeighbors(n_neighbors=2, metric='euclidean')

최근접 이웃을 구하는 kneighbors 메서드에서는 n_neighbors의 매개변수를 다르게 지정할 수 있으며 인덱스만 반환할 수 있다.

In [19]:
indicies = nearest_neighbors.kneighbors([new], n_neighbors=5, return_distance=False)

반면에, 반경을 설정하면 주어진 반경 내에 위치한 이웃을 모두 찾아주는 방법도 있다. 반경을 NearestNeighbors 클래스의 radius에 지정해도 되며 radius_neighbors method에서도 radius 값을 설정할 수 있다.

In [22]:
nearest_neighbors = NearestNeighbors(n_neighbors=2, radius=0.5).fit(features_standardized)

In [23]:
indicies = nearest_neighbors.radius_neighbors([new], radius=0.5, return_distance=False)

### k-최근접 이웃 분류기 만들기
클래스를 모르는 샘플이 주어졌을 대 이웃한 샘플의 클래스를 기반으로 이 샘플의 클래스를 예측해야 한다. -> 데이터셋이 크지 않다면 KNeighborsClassifier를 사용한다.

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

# 데이터
iris = datasets.load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1).fit(X_std, y)

new_observations = [[.75, .75, .75, .75], [1, 1, 1, 1]]
print(knn.predict(new_observations))
print(knn.predict_proba(new_observations))

[1 2]
[[0.  0.6 0.4]
 [0.  0.  1. ]]


- 샘플이 주어지면 KNN알고리즘은 먼저 거리 측정 방법을 기반으로 가장 가까운 k개의 샘플을 찾는다. 그다음 k개 샘플의 클래스를 기반으로 투표를 한다. 가장 많은 표를 얻은 클래스가 샘플의 예측 클래스가 된다. 샘플이 클래스 j일 확률은  
    
    $\frac{1}{k}\sum_{i \in{v}} I (y_i=j)$  
    ($v$는 $k$개의 샘플, $y$는 $i$번째 샘플의 클래스. $I$는 지시 함수(1은 참, 0은 그외))
    
    
- KNeighborsClassifier의 중요 매개변수
    1. metric: 거리 측정 방법 지정.
    2. n_jobs: 얼마나 많은 컴퓨터 코어를 사용할지 결정. 예측을 하려면 한 샘플에서 데이터의 모든 샘플까지 거리를 계산해야 해서 여러 개를 사용하는 것이 좋다.
    3. algorithm: 가장 가까운 이웃을 계산하기 위한 방법 지정. 그러나 KNeighborsClassifier가 자동으로 찾는다.
    4. weights: distance로 지정하면 가까운 이웃의 투표에 가중치를 더 부여한다.


KNeighborsRegressor는 회귀문제에 사용하는 k-최근접 이웃 분류기이다. 회귀 문제에서 사용하면 될 듯 하다.

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets

boston = datasets.load_boston()
features = boston.data[:,0:2]
target = boston.target

knn_regressor = KNeighborsRegressor(n_neighbors=10)

model1 = knn_regressor.fit(features, target)
print(model1.predict(features[0:1])[0]*1000)

# KNeighborsClassifier와 KNeighborsRegressor 모두 NearestNeighbors 클래스를 상속해 kneighbors method를 사용해서 최근접 이웃을 확인할 수 있다.

32440.000000000004


### 최선의 이웃 개수 결정하기.
k-최근접 이웃 분류기에서 최선의 k값을 찾아야 한다. -> GridSearchCV와 같은 모델 선택 기법을 사용.

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

iris = datasets.load_iris()
features = iris.data
target = iris.target

# 표준화 객체 생성.
standardizer = StandardScaler()

# KNN 분류기 생성.
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

# 파이프 라인 생성. 추후 그리드 서치에 입력할 것.
pipe = Pipeline([("standardizer", standardizer), ("knn", knn)])

# 탐색 영역의 후보 생성. 추후 그리드 서치에 입력할 것.
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

# 그리드 서치
classifier = GridSearchCV(pipe, search_space, cv=5, verbose=0).fit(features, target)

# 최선의 이웃 개수 출력
classifier.best_estimator_.get_params()["knn__n_neighbors"]

6

k는 편향과 분산에 큰 영향을 끼친다. n이 샘플의 개수일 때 k=n이면 편향이 높고 분산이 낮다. k=1이면 편향이 낮고 분산이 높다. 이 편향-분산 트레이드오프의 균형을 맞추는 k값을 찾아야만 최선의 모델을 만들 수 있고 이는 GridSearchCV를 통해 할 수 있다.